In [36]:
import pysindy as ps

import deepSI
from deepSI.fit_systems import SS_encoder_general
from deepSI.fit_systems.encoders import default_encoder_net, default_state_net, default_output_net

import torch
from torch import nn

import numpy as np

from sklearn.preprocessing import PolynomialFeatures

In [37]:
class SS_encoder_general_eq(SS_encoder_general):
    def __init__(self, nx=10, na=20, nb=20, feedthrough=False, \
        e_net=default_encoder_net, f_net=default_state_net, h_net=default_output_net, \
        e_net_kwargs={},           f_net_kwargs={},         h_net_kwargs={}, na_right=0, nb_right=0, \
        gamma=1e-4):

        super(SS_encoder_general_eq, self).__init__()
        self.nx, self.na, self.nb = nx, na, nb
        self.k0 = max(self.na,self.nb)
        
        self.e_net = e_net
        self.e_net_kwargs = e_net_kwargs

        self.f_net = f_net
        self.f_net_kwargs = f_net_kwargs

        self.h_net = h_net
        self.h_net_kwargs = h_net_kwargs

        self.feedthrough = feedthrough
        self.na_right = na_right
        self.nb_right = nb_right
        ######################################
        # args added for feature transform and
        # regurlarization
        self.gamma = gamma
        ######################################

    def init_nets(self, nu, ny): # a bit weird
        na_right = self.na_right if hasattr(self,'na_right') else 0
        nb_right = self.nb_right if hasattr(self,'nb_right') else 0
        self.encoder = self.e_net(nb=(self.nb+nb_right), nu=nu, na=(self.na+na_right), ny=ny, nx=self.nx, **self.e_net_kwargs)
        ######################################
        ###### change fn intialization #######
        self.fn     =      self.f_net(nx=self.nx, nu=nu, **self.f_net_kwargs)
        ######################################
        if self.feedthrough:
            self.hn =      self.h_net(nx=self.nx, ny=ny, nu=nu,                     **self.h_net_kwargs) 
        else:
            self.hn =      self.h_net(nx=self.nx, ny=ny,                            **self.h_net_kwargs) 

    def loss(self, uhist, yhist, ufuture, yfuture, loss_nf_cutoff=None, **Loss_kwargs):
        x = self.encoder(uhist, yhist) #initialize Nbatch number of states
        errors = []
        
        for y, u in zip(torch.transpose(yfuture,0,1), torch.transpose(ufuture,0,1)): #iterate over time
            error = nn.functional.mse_loss(y, self.hn(x,u) if self.feedthrough else self.hn(x))
            ##################################
            ## add penalty to weights in fn ##
            params = [*self.fn.parameters()]
            weights = [x.view(-1) for x in params][0]
            error += self.gamma*torch.norm(weights, 1)
            ##################################
            errors.append(error) #calculate error after taking n-steps
            if loss_nf_cutoff is not None and error.item()>loss_nf_cutoff:
                print(len(errors), end=' ')
                break
            x = self.fn(x,u) #advance state. 
            
        return torch.mean(torch.stack(errors))

In [38]:
class identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

    def forward(self, input):
        return input[:,-1]
    
class simple_Linear(torch.nn.Module):
    def __init__(self, nx, nu, **kwargs):
        super(simple_Linear, self).__init__()

        self.nx = nx
        self.nu = kwargs['u']

        self.feature_library = kwargs['feature_library']
        test_sample = torch.rand(1,self.nx+self.nu, requires_grad=True)
        self.nf = (self.feature_library.fit_transform(test_sample)).shape[1]
        
        self.layer = nn.Linear(self.nf, nx, bias=False)
        

    def forward(self, x, u):
        x = torch.hstack((x, u.unsqueeze(1)))
        Theta = self.feature_library.fit_transform(x)
        mu = torch.mean(Theta)
        std = torch.std(Theta)
        Theta = (Theta-mu)/std
        out = self.layer(Theta)
        return out

In [39]:
class feature_library():
    def __init__(
            self,
            functions,
            interaction_only=True
    ):
        self.functions = functions
        self.interaction_only = interaction_only

    def fit_transform(self, X):
        # off set
        out_feature = ((X[:,0])**0).unsqueeze(1)
        if self.interaction_only:
            for f in self.functions:
                out_feature = torch.hstack((out_feature, f(X)))
            return out_feature
        

In [40]:
def f(x):
  return x

def f2(x):
  return x**2

def f3(x):
  return x**3

In [41]:
# functions = [lambda x:x,
#              lambda x:x**2,
#              lambda x:x**3]
functions = [f, f2, f3]

poly = feature_library(functions=functions)

In [46]:
# fit_sys = deepSI.fit_systems.SS_encoder_general(nx=2, na=50, nb=50)

nx, nu = 2, 1 # state dimension and inputs
na, nb = 5, 5

f_net = simple_Linear
f_net_kwargs= f_net_kwargs={"feature_library": poly, "u": nu, "nf": 10}

h_net = identity
h_net_kwargs = {}

fit_sys = SS_encoder_general_eq(nx=2, na=50, nb=50, \
                                f_net=f_net, f_net_kwargs=f_net_kwargs,\
                                h_net=identity)

train, test = deepSI.datasets.Silverbox()
train, test = train[:1000], test[:1000]

fit_sys.fit(train, test, epochs=20, batch_size = 2, optimizer_kwargs={"lr": 1e-7}, loss_kwargs=dict(nf=100))

Initilizing the model and optimizer
Size of the training array =  1.9 MB
N_training_samples = 851, batch_size = 2, N_batch_updates_per_epoch = 425
Initial Validation sim-NRMS= 1.0233330686621502


 50%|█████     | 1/2 [00:38<00:38, 38.21s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 1.0232632284580911
Epoch    1, sqrt loss   1.052, Val sim-NRMS  1.023, Time Loss: 99.1%, data: 0.3%, val: 0.6%, 11.1 batches/sec


100%|██████████| 2/2 [01:16<00:00, 38.10s/it]

########## New lowest validation loss achieved ########### sim-NRMS = 1.0231937520263894
Epoch    2, sqrt loss   1.051, Val sim-NRMS  1.023, Time Loss: 99.1%, data: 0.2%, val: 0.6%, 11.2 batches/sec
Loaded model with best known validation sim-NRMS of  1.023 which happened on epoch 2 (epoch_id=2.00)


In [44]:
[*fit_sys.fn.parameters()]

[Parameter containing:
 tensor([[-0.0614,  0.1140, -0.0476,  0.1463,  0.1167,  0.1051,  0.1902,  0.2439,
           0.1115,  0.0896],
         [-0.1570,  0.2194,  0.0225,  0.1736,  0.2099, -0.2865, -0.2864,  0.1420,
          -0.1513, -0.2849]], requires_grad=True)]